In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random

In [60]:
ns4 = pd.read_pickle("DataStore/2018-08/new_related_field_pats_0918.pkl").drop(['tp_msa_pc_pats',
       'op_msa_pc_pats'],1)
print(len(ns4))
ns4 = ns4.dropna(how="any")
print(len(ns4))

ns4["years_from_first"] = ns4["gyear"]-ns4["tp_first_gyear"]

41297
40914


In [61]:
n2 = ns4[["tp_msa_pc", "op_msa_pc", "msa_pc_docvecs"]].groupby(["tp_msa_pc", "op_msa_pc",]).mean().reset_index()
n2["tp_inv_msa"] = n2["tp_msa_pc"].apply(lambda x: x[0])
n2["tp_primclass"] = n2["tp_msa_pc"].apply(lambda x: x[1])
n2["op_inv_msa"] = n2["op_msa_pc"].apply(lambda x: x[0])
n2["op_primclass"] = n2["op_msa_pc"].apply(lambda x: x[1])
n2["inv_msa_match"] = (n2["tp_inv_msa"] == n2["op_inv_msa"])

self_sim = n2.loc[n2["inv_msa_match"] == True]
self_sim = dict(zip(self_sim["tp_msa_pc"], self_sim["msa_pc_docvecs"]))

# Self similarity
n2["same_msa_docvecs"] = n2["tp_msa_pc"].map(self_sim)
n2["diff_docvecs"] = n2["msa_pc_docvecs"]-n2["same_msa_docvecs"]

In [62]:
more_sim = n2.loc[(n2["diff_docvecs"] > 0)]
less_sim = n2.loc[(n2["diff_docvecs"] < 0)]

more_prop = more_sim["tp_primclass"].value_counts()/more_sim["tp_primclass"].value_counts().sum()
less_prop = less_sim["tp_primclass"].value_counts()/less_sim["tp_primclass"].value_counts().sum()
display(more_prop)
display(less_prop)

709.0    0.286765
707.0    0.187500
1.0      0.125000
713.0    0.110294
726.0    0.080882
717.0    0.055147
725.0    0.044118
716.0    0.044118
718.0    0.018382
800.0    0.018382
703.0    0.011029
719.0    0.011029
706.0    0.007353
Name: tp_primclass, dtype: float64

709.0    0.369535
1.0      0.136812
726.0    0.121298
713.0    0.110014
707.0    0.095910
717.0    0.052186
800.0    0.028209
725.0    0.025388
716.0    0.021157
719.0    0.018336
706.0    0.008463
718.0    0.007052
703.0    0.005642
Name: tp_primclass, dtype: float64

In [63]:
len(more_sim)

272

In [64]:
len(less_sim)

709

In [45]:
n2.loc[(n2["diff_docvecs"] < 0)]

,tp_msa_pc,op_msa_pc,msa_pc_docvecs,tp_inv_msa,tp_primclass,op_inv_msa,op_primclass,inv_msa_match,same_msa_docvecs,diff_docvecs
1,"(Albany-Schenectady-Troy, NY, 382.0)","(Boston-Cambridge-Quincy, MA-NH, 382.0)",0.185140,"Albany-Schenectady-Troy, NY",382.0,"Boston-Cambridge-Quincy, MA-NH",382.0,False,0.247386,-0.062246
2,"(Albany-Schenectady-Troy, NY, 382.0)","(Chicago-Joliet-Naperville, IL-IN-WI, 382.0)",0.208694,"Albany-Schenectady-Troy, NY",382.0,"Chicago-Joliet-Naperville, IL-IN-WI",382.0,False,0.247386,-0.038692
3,"(Albany-Schenectady-Troy, NY, 382.0)","(New York-Northern New Jersey-Long Island, NY-...",0.182632,"Albany-Schenectady-Troy, NY",382.0,"New York-Northern New Jersey-Long Island, NY-N...",382.0,False,0.247386,-0.064753
4,"(Albany-Schenectady-Troy, NY, 382.0)","(Rochester, NY, 382.0)",0.192443,"Albany-Schenectady-Troy, NY",382.0,"Rochester, NY",382.0,False,0.247386,-0.054942
5,"(Albany-Schenectady-Troy, NY, 382.0)","(San Francisco-Oakland-Fremont, CA, 382.0)",0.184885,"Albany-Schenectady-Troy, NY",382.0,"San Francisco-Oakland-Fremont, CA",382.0,False,0.247386,-0.062501
6,"(Albany-Schenectady-Troy, NY, 382.0)","(San Jose-Sunnyvale-Santa Clara, CA, 382.0)",0.164926,"Albany-Schenectady-Troy, NY",382.0,"San Jose-Sunnyvale-Santa Clara, CA",382.0,False,0.247386,-0.082459
8,"(Albany-Schenectady-Troy, NY, 600.0)","(San Jose-Sunnyvale-Santa Clara, CA, 600.0)",0.159968,"Albany-Schenectady-Troy, NY",600.0,"San Jose-Sunnyvale-Santa Clara, CA",600.0,False,0.206758,-0.046790
9,"(Albany-Schenectady-Troy, NY, 600.0)","(Seattle-Tacoma-Bellevue, WA, 600.0)",0.161820,"Albany-Schenectady-Troy, NY",600.0,"Seattle-Tacoma-Bellevue, WA",600.0,False,0.206758,-0.044938
11,"(Albuquerque, NM, 356.0)","(Philadelphia-Camden-Wilmington, PA-NJ-DE-MD, ...",0.189774,"Albuquerque, NM",356.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",356.0,False,0.270796,-0.081022
16,"(Allentown-Bethlehem-Easton, PA-NJ, 327.0)","(San Jose-Sunnyvale-Santa Clara, CA, 327.0)",0.202661,"Allentown-Bethlehem-Easton, PA-NJ",327.0,"San Jose-Sunnyvale-Santa Clara, CA",327.0,False,0.231172,-0.028511


In [22]:
print(len(ns5.loc[ns5["msa_pc_pats_docvecs"] > ns5["tp_msa_pc_pats_docvecs"]]))
display(ns5.loc[ns5["msa_pc_pats_docvecs"] > ns5["tp_msa_pc_pats_docvecs"]])

94


,msa_pc_pats_docvecs,tp_msa_pc_pats_docvecs
tp_msa_pc,,
"(Albany-Schenectady-Troy, NY, 705.0)",0.171943,0.161318
"(Ames, IA, 800.0)",0.227102,0.214132
"(Atlanta-Sandy Springs-Marietta, GA, 382.0)",0.166607,0.163486
"(Austin-Round Rock-San Marcos, TX, 320.0)",0.170768,0.170461
"(Austin-Round Rock-San Marcos, TX, 353.0)",0.295281,0.122277
"(Austin-Round Rock-San Marcos, TX, 430.0)",0.213461,0.177804
"(Austin-Round Rock-San Marcos, TX, 726.0)",0.140824,0.131802
"(Baltimore-Towson, MD, 713.0)",0.171518,0.157238
"(Binghamton, NY, 705.0)",0.119643,0.118887


In [21]:
print(len(ns5.loc[ns5["msa_pc_pats_docvecs"] <= ns5["tp_msa_pc_pats_docvecs"]]))

94


In [24]:
ns4

,gyear,op_msa_pc,tp_first_gyear,tp_msa_pc,msa_pc_pats_docvecs,tp_msa_pc_pats_docvecs
0,1991,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",0.226491,0.226630
1,1996,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",0.205869,0.168002
2,1997,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",0.226116,0.171872
3,1998,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",0.240150,0.219993
4,1999,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",0.079426,0.046552
5,2000,"(San Jose-Sunnyvale-Santa Clara, CA, 370.0)",1991,"(Kansas City, MO-KS, 370.0)",0.189254,0.185522
6,1996,"(San Jose-Sunnyvale-Santa Clara, CA, 709.0)",1996,"(Raleigh-Cary, NC, 707.0)",0.150804,0.141067
7,1999,"(San Jose-Sunnyvale-Santa Clara, CA, 709.0)",1996,"(Raleigh-Cary, NC, 707.0)",0.098048,0.130681
8,2000,"(San Jose-Sunnyvale-Santa Clara, CA, 709.0)",1996,"(Raleigh-Cary, NC, 707.0)",0.131366,0.125395
9,2001,"(San Jose-Sunnyvale-Santa Clara, CA, 709.0)",1996,"(Raleigh-Cary, NC, 707.0)",0.233463,0.211442
